In [ ]:
from pyscipopt import Model, quicksum
import numpy as np
import pickle

rnd = np.random
models_output = []
for k in range(800):
    info = {}
    np.random.seed(k)
    n_dc = 100
    n_cus = 200
    c = []
    loc_x_dc = rnd.rand(n_dc)*100
    loc_y_dc = rnd.rand(n_dc)*100
    loc_x_cus = rnd.rand(n_cus)*100
    loc_y_cus = rnd.rand(n_cus)*100
    xy_dc = [[x,y] for x,y in zip(loc_x_dc,loc_y_dc)]
    xy_cus = [[x,y] for x,y in zip(loc_x_cus,loc_y_cus)]
    for i in xy_dc:
        c_i = []
        for j in xy_cus:
            c_i.append(np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)*0.01)
        c.append(c_i)    
    
    f= []
    m = []
    d = []
    
    for i in range(n_dc):
        f_rand = np.round(np.random.normal(100,15))
        if f_rand < 40:
            f.append(40)
        else:
            f.append(f_rand)
        m_rand = np.round(np.random.normal(70,10))
        if m_rand < 30:
            m.append(30)
        else:
            m.append(m_rand)
    for i in range(n_cus):
        d_rand = np.round(np.random.normal(20,5))
        if d_rand < 5:
            d.append(5)
        else:
            d.append(d_rand)
    
    f = np.array(f)
    m = np.array(m)
    d = np.array(d)
    c = np.array(c)

    model = Model("facility selection")
    x,y = {},{}
    x_names = ['x_'+str(i)+'_'+str(j) for i in range(n_dc) for j in range(n_cus)]
    y_names = ['y_'+str(i) for i in range(n_dc)]
    for i in range(n_dc):
        for j in range(n_cus):
            x[i,j] = model.addVar(name=x_names[i*n_cus+j])
    for i in range(n_dc):
        y[i] = model.addVar(name=y_names[i],vtype='BINARY')
    model.setObjective(quicksum(f[i]*y[i] for i in range(n_dc))+quicksum(c[i,j]*x[i,j] for i in range(n_dc) for j in range(n_cus)), 'minimize')
    for j in range(n_cus):
        model.addCons(quicksum(x[i,j] for i in range(n_dc)) == d[j])
    for i in range(n_dc):
        model.addCons(quicksum(x[i,j] for j in range(n_cus)) <= m[i]*y[i])
    model.optimize()
    
    y_sol = []
    for i in range(n_dc):
        y_sol.append(model.getVal(y[i]))
    x_sol = []
    for i in range(n_dc):
        x_i = []
        for j in range(n_cus):
            x_i.append(model.getVal(x[i,j]))
        x_sol.append(x_i)
    obj_vol = model.getObjVal()
    
    info['f'] = f
    info['m'] = m
    info['d'] = d
    info['c'] = c
    info['y'] = y_sol
    info['x'] = x_sol
    info['obj'] = obj_vol
    info['xy_dc'] = xy_dc
    info['xy_cus'] = xy_cus
    models_output.append(info)

with open('models_output.pkl', 'wb') as outp:
    pickle.dump(models_output, outp)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
def convert_coord(xy):
    r = np.sqrt((xy[0]-50)**2 + (xy[1]-50)**2)
    theta = np.arctan2(xy[1],xy[0])
    return [theta, r]

dataset = []
for k in range(len(models_output)):
    data = {}
    xy_site = []
    new_xy_dc = [convert_coord(i) for i in models_output[k]['xy_dc']]
    new_xy_cus = [convert_coord(i) for i in models_output[k]['xy_cus']]
    xy_site.extend(new_xy_dc)
    xy_site.extend(new_xy_cus)
    d_site = []
    d_site.extend([[i] for i in models_output[k]['m']])
    d_site.extend([[i] for i in models_output[k]['d']])
    f_site = []
    f_site.extend([[i] for i in models_output[k]['f']])
    f_site.extend([[0] for i in range(200)])
    i_site = []
    i_site.extend([[0] for i in range(100)])
    i_site.extend([[1] for i in range(200)])
    x = np.concatenate((xy_site, d_site), axis=1)
    x = np.concatenate((x, f_site), axis=1)
    x = np.concatenate((x, i_site), axis=1)
    if k == 0:
        scaler_x = MinMaxScaler()
        x = scaler_x.fit_transform(x)
    else:
        x = scaler_x.transform(x)
    x = np.expand_dims(x,axis=1)
    y = []
    y_cus = [2 for i in range(200)]
    y.extend(models_output[k]['y'])
    y.extend(y_cus)
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(np.array(y)).long()
    data['x'] = x
    data['y'] = y
    dataset.append(data)
    mask = []
    mask_true = [True for i in range(100)]
    mask_false = [False for i in range(200)]
    mask.extend(mask_true)
    mask.extend(mask_false)

In [ ]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerModel(nn.Module):
    def __init__(self, n_class=2, d_input=5, d_model=256, nhead=8, d_hid=256, nlayers=3, dropout=0.5):
        super().__init__()
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.d_model = d_model
        self.encoder = nn.Linear(d_input, d_model)
        self.decoder = nn.Linear(d_model, n_class)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.decoder.weight.data.uniform_(-initrange, initrange)
   
  
    def forward(self, src: Tensor) -> Tensor:
        src = self.encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

In [ ]:
import torch.optim as optim

def acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    return acc

LEARNING_RATE = 0.0001
EPOCHS = 100
model = TransformerModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
n_class = 2

for e in range(1,EPOCHS+1):
    model.train()
    e_train_loss = []
    e_train_acc = []
    for i in range(560):
        optimizer.zero_grad()
        y_pred = model(dataset[i]['x'])
        train_loss = criterion(y_pred.view(-1,n_class)[mask], dataset[i]['y'][mask])
        train_acc = acc(y_pred.view(-1,n_class)[mask], dataset[i]['y'][mask])
        train_loss.backward()
        optimizer.step()
        e_train_loss.append(train_loss.item())
        e_train_acc.append(train_acc.item())
    with torch.no_grad():
        model.eval()
        e_val_loss = []
        e_val_acc = []
        for i in range(560,800):
            y_pred = model(dataset[i]['x'])
            val_loss = criterion(y_pred.view(-1,n_class)[mask], dataset[i]['y'][mask])
            val_acc = acc(y_pred.view(-1,n_class)[mask], dataset[i]['y'][mask])
            e_val_loss.append(val_loss.item())
            e_val_acc.append(val_acc.item())
    print('epoch:', e)
    print('train loss:', np.mean(e_train_loss))
    print('train acc:', np.mean(e_train_acc)) 
    print('val loss:', np.mean(e_val_loss))
    print('val acc:', np.mean(e_val_acc))

torch.save(model.state_dict(), desired_path)

In [ ]:
import time
from bisect import bisect

def resolve_infeasibility(m,f,d,y):
    idx = np.arange(len(y))
    idx = idx[y==0]
    m = m[idx]
    f = f[idx]
    r = f/m
    r_idx = np.argsort(r)
    m_sort = m[r_idx]
    idx = idx[r_idx]
    m_sort_sum = np.cumsum(m_sort)
    up_to_idx = bisect(m_sort_sum,d)
    idx = idx[:up_to_idx+1]
    for i in idx:
        y[i] = 1
    return y

transformer_model = TransformerModel()
transformer_model.load_state_dict(torch.load(desired_path))

obj_transformer = []
gap_transformer = []
time_transformer = []
obj_original = []
gap_original = []
time_original = []

rnd = np.random

for k in range(800,900):
    print(k)
    np.random.seed(k)
    n_dc = 100
    n_cus = 200
    c = []
    loc_x_dc = rnd.rand(n_dc)*100
    loc_y_dc = rnd.rand(n_dc)*100
    loc_x_cus = rnd.rand(n_cus)*100
    loc_y_cus = rnd.rand(n_cus)*100
    xy_dc = [[x,y] for x,y in zip(loc_x_dc,loc_y_dc)]
    xy_cus = [[x,y] for x,y in zip(loc_x_cus,loc_y_cus)]
    for i in xy_dc:
        c_i = []
        for j in xy_cus:
            c_i.append(np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)*0.01)
        c.append(c_i)    
    
    f= []
    m = []
    d = []
    
    for i in range(n_dc):
        f_rand = np.round(np.random.normal(100,15))
        if f_rand < 40:
            f.append(40)
        else:
            f.append(f_rand)
        m_rand = np.round(np.random.normal(70,10))
        if m_rand < 30:
            m.append(30)
        else:
            m.append(m_rand)
    for i in range(n_cus):
        d_rand = np.round(np.random.normal(20,5))
        if d_rand < 5:
            d.append(5)
        else:
            d.append(d_rand)
    
    f = np.array(f)
    m = np.array(m)
    d = np.array(d)
    c = np.array(c)
    
    start_time = time.time()
    model = Model("facility selection")
    x,y = {},{}
    x_names = ['x_'+str(i)+'_'+str(j) for i in range(n_dc) for j in range(n_cus)]
    y_names = ['y_'+str(i) for i in range(n_dc)]
    for i in range(n_dc):
        for j in range(n_cus):
            x[i,j] = model.addVar(name=x_names[i*n_cus+j])
    for i in range(n_dc):
        y[i] = model.addVar(name=y_names[i],vtype='BINARY')
    model.setObjective(quicksum(f[i]*y[i] for i in range(n_dc))+quicksum(c[i,j]*x[i,j] for i in range(n_dc) for j in range(n_cus)), 'minimize')
    for j in range(n_cus):
        model.addCons(quicksum(x[i,j] for i in range(n_dc)) == d[j])
    for i in range(n_dc):
        model.addCons(quicksum(x[i,j] for j in range(n_cus)) <= m[i]*y[i])
    model.optimize()
    time_original.append(time.time()-start_time)
    obj_original.append(model.getObjVal())
    gap_original.append(model.getGap())
    
    start_time = time.time()
    xy_site = []
    new_xy_dc = [convert_coord(i) for i in xy_dc]
    new_xy_cus = [convert_coord(i) for i in xy_cus]
    xy_site.extend(new_xy_dc)
    xy_site.extend(new_xy_cus)
    d_site = []
    d_site.extend([[i] for i in m])
    d_site.extend([[i] for i in d])
    f_site = []
    f_site.extend([[i] for i in f])
    f_site.extend([[0] for i in range(200)])
    i_site = []
    i_site.extend([[0] for i in range(100)])
    i_site.extend([[1] for i in range(200)])
    x = np.concatenate((xy_site, d_site), axis=1)
    x = np.concatenate((x, f_site), axis=1)
    x = np.concatenate((x, i_site), axis=1)
    x = scaler_x.transform(x)
    x = np.expand_dims(x,axis=1)
    x = torch.from_numpy(x).float()
    
    transformer_model.eval()
    y_pred = transformer_model(x)
    y_pred_softmax = torch.log_softmax(y_pred.view(-1,2)[mask], dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
    if np.sum(m*y_pred_tags.numpy()) < np.sum(d):
        y_pred_corrected = resolve_infeasibility(m,f,np.sum(d)-np.sum(m*y_pred_tags.numpy()),y_pred_tags.numpy())
    else:
        y_pred_corrected = y_pred_tags.numpy()
    
    model = Model("facility selection with transformer")
    x,y = {},{}
    x_names = ['x_'+str(i)+'_'+str(j) for i in range(n_dc) for j in range(n_cus)]
    y_names = ['y_'+str(i) for i in range(n_dc)]
    for i in range(n_dc):
        for j in range(n_cus):
            x[i,j] = model.addVar(name=x_names[i*n_cus+j])
    
    for i in range(n_dc):
        if y_pred_corrected[i] == 1:
            y[i] = model.addVar(name=y_names[i],vtype='BINARY',lb=1)
        else:
            y[i] = model.addVar(name=y_names[i],vtype='BINARY',ub=0)
    model.setObjective(quicksum(f[i]*y[i] for i in range(n_dc))+quicksum(c[i,j]*x[i,j] for i in range(n_dc) for j in range(n_cus)), 'minimize')
    for j in range(n_cus):
        model.addCons(quicksum(x[i,j] for i in range(n_dc)) == d[j])
    for i in range(n_dc):
        model.addCons(quicksum(x[i,j] for j in range(n_cus)) <= m[i]*y[i])
    model.optimize()
    time_transformer.append(time.time()-start_time)
    obj_transformer.append(model.getObjVal())
    gap_transformer.append(model.getGap())